### 행정안전부 보도자료 데이터 수집
- 행정안전부 : https://www.mois.go.kr/frt/a01/frtMain.do
- 정적페이지 데이터 수집 : html : bs, css-selector
- post 방식
- 여러개의 페이지 데이터 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 1. 웹서비스 분석 : url

#### 멀티라인 코드 수정 TIP
1. alt
2. ctrl+오른쪽키
3. 오른쪽키두번
4. end키

In [4]:
url = "https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": "3",
    "cal_url": "/sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
}

In [5]:
# 2. request(url, params) > response : html

In [6]:
response = requests.post(url, params)
response

<Response [200]>

In [8]:
response.text[:500]

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<title>\r\n\r\n행정안전부&gt;  뉴스·소식&gt; 보도자료&gt; 보도자료</title>\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<meta charset="utf-8" />\r\n<meta name="viewport" content="width=device-width,maximum-scale=1,minimum-scale=1,user-scalable=no" />\r\n<meta name="author" content="행정안전부" />\r\n<meta name="subject" content="행정안전부" />\r\n<meta name="keywords" content="행정안전부, 행안부, 정부혁신, 지방자치, 생활자치, 규제개혁, 지역경제 활성화, 국민생활 불편해소, 지방재정 개혁, 행정관리, 디지털 정부, 복지서비스, 복지사각지대, 지방공기업 경쟁'

In [25]:
# 3. html(str) > bs_obj > bs_obj.select(css-selector) > text > DataFrame

In [26]:
dom = BeautifulSoup(response.text, "html.parser")

In [27]:
# 게시글 리스트 데이터 수집 : 10개
elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
len(elements)

10

In [28]:
# 각 데이터에서 필요한 정보 수집
element = elements[0]
data = {
    #strip() : 앞뒤 공백 문자 제거
    "no":element.select("td")[0].text.strip(),
    "title":element.select("td")[1].text.strip(),
    "writer":element.select("td")[3].text.strip(),
    "date":element.select("td")[4].text.strip(),
    "page_view":element.select("td")[5].text.strip(),
    # 속성값을 가져올땐 .get()
    "link":element.select("td")[1].select_one("a").get("href"),
}

data

{'no': '12931',
 'title': '사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다',
 'writer': '공공지능정책과',
 'date': '2022.02.15.',
 'page_view': '812',
 'link': '/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=moq-VM7jEOtPaTDppsyZv-a6.node20?bbsId=BBSMSTR_000000000008&nttId=90455'}

In [31]:
# [{row1}, {row2}, ...] 형태의 데이터프레임으로 만들어주기
data = []
for element in elements:
    data.append({
        "no":element.select("td")[0].text.strip(),
        "title":element.select("td")[1].text.strip(),
        "writer":element.select("td")[3].text.strip(),
        "date":element.select("td")[4].text.strip(),
        "page_view":element.select("td")[5].text.strip(),
        # 속성값을 가져올땐 .get()
        "link":element.select("td")[1].select_one("a").get("href"),
    })
df = pd.DataFrame(data)
df.tail(2)

,no,title,writer,date,page_view,link
8,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1042,/frt/bbs/type010/commonSelectBoardArticle.do;j...
9,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1120,/frt/bbs/type010/commonSelectBoardArticle.do;j...


In [29]:
# 4. function : params : page(1번페이지, 2번페이지, ...)

In [32]:
def mois(page):
    # 1. 웹서비스 분석 : url
    url = "https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
    params = {
        "nttId": "0",
        "bbsTyCode": "BBST03",
        "bbsAttrbCode": "BBSA03",
        "authFlag": "Y",
        "pageIndex": "3",
        "cal_url": "/sym/cal/EgovNormalCalPopup.do",
        "searchCnd": "0",
    }
    # 2. request(url, params) > response : html
    response = requests.post(url, params)
    
    # 3. html(str) > bs_obj > bs_obj.select(css-selector) > text > DataFrame
    dom = BeautifulSoup(response.text, "html.parser")
    # 게시글 리스트 데이터 수집 : 10개
    elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
    # 각 데이터에서 필요한 정보 수집
    data = []
    for element in elements:
        data.append({
            "no":element.select("td")[0].text.strip(),
            "title":element.select("td")[1].text.strip(),
            "writer":element.select("td")[3].text.strip(),
            "date":element.select("td")[4].text.strip(),
            "page_view":element.select("td")[5].text.strip(),
            # 속성값을 가져올땐 .get()
            "link":element.select("td")[1].select_one("a").get("href"),
        })
    df = pd.DataFrame(data)
    
    
    return df

In [33]:
mois(2)

,no,title,writer,date,page_view,link
0,12931,사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다,공공지능정책과,2022.02.15.,825,/frt/bbs/type010/commonSelectBoardArticle.do;j...
1,12930,"2022 지방행정혁신, 주민이 체감하는 확실한 변화 만든다",지역사회혁신정책과,2022.02.15.,682,/frt/bbs/type010/commonSelectBoardArticle.do;j...
2,12929,적극행정을 일상적 공직문화로 자리매김하여 국민이 현장에서 체감하는 성과를 창출하겠습니다,지역사회혁신정책과,2022.02.15.,560,/frt/bbs/type010/commonSelectBoardArticle.do;j...
3,12928,'가명정보 결합전문기관 실무협의회' 출범,공공데이터정책과,2022.02.14.,649,/frt/bbs/type010/commonSelectBoardArticle.do;j...
4,12927,코로나19로 지친 국민들에게 지방세 지원을 강화한다,지방세정책과,2022.02.14.,947,/frt/bbs/type010/commonSelectBoardArticle.do;j...
5,12926,"행정안전부 소통협력공간, 인구감소지역 최초 선정",지역사회혁신정책과,2022.02.13.,955,/frt/bbs/type010/commonSelectBoardArticle.do;j...
6,12925,국민이 제안하고 풀어가는 민생규제 혁신과제 공모전 개최,지방규제혁신과,2022.02.13.,789,/frt/bbs/type010/commonSelectBoardArticle.do;j...
7,12924,이제 스마트폰으로 주민등록증 확인 가능해진다... 4월 시범실시 예정,주민과,2022.02.10.,1762,/frt/bbs/type010/commonSelectBoardArticle.do;j...
8,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1044,/frt/bbs/type010/commonSelectBoardArticle.do;j...
9,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1122,/frt/bbs/type010/commonSelectBoardArticle.do;j...


In [ ]:
# 5. 여러 페이지 데이터 수집 : pandas.concat()사용

In [35]:
dfs = []
for page in range(1,4):
    print(page, end=" ")
    # 1페이지 수집해서 dfs에 저장하면 dfs에는 3페이지에서 수집된 데이터가 들어감
    dfs.append(mois(page)) 

1 2 3 

In [ ]:
# 1페이지
dfs[0] 
# 2페이지
dfs[1]
# 3페이지
dfs[2]

In [36]:
result_df = pd.concat(dfs, ignore_index=True)
result_df.tail(2)

,no,title,writer,date,page_view,link
28,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1044,/frt/bbs/type010/commonSelectBoardArticle.do;j...
29,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1122,/frt/bbs/type010/commonSelectBoardArticle.do;j...
